## Crawl and Scrape all info from the category: drugs

In [9]:
# imports
import socks
import socket
import requests
from bs4 import BeautifulSoup
import time
import re
import numpy as np
import pandas as pd

In [23]:
# defaults
socks.set_default_proxy(socks.SOCKS5, "localhost", 1080) # one source said port 9150, Georgia's code used 9050
socket.socket = socks.socksocket

# These lines are copied from the Linkedin post.
def getaddrinfo(*args):
    return [(socket.AF_INET, socket.SOCK_STREAM, 6, '', (args[0], args[1]))]
socket.getaddrinfo = getaddrinfo


s = requests.Session()
s.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0'
})

proxies = {
    'http': 'socks5h://localhost:9050',
    'https': 'socks5h://localhost:9050'
}



In [18]:

# ToRReZ market URL DEZE LINKE GEBRUIKEN OM DE COOCKIES TE PAKKEN
url = "http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/category/drugs-and-chemicals?order=none&vendors=all&shipping_to=any&shipping_from=any&product_type=any&payments_method=any&page=1"
url = "http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/category/drugs-and-chemicals"


In [24]:

# Manually selected from the browser header after solving the captcha
# dev tools > network > select get request > headers > scroll to request header > copy "Cookie"

# cookies_copied = "Cookie: PHPSESSID=kfv1msv5hilf0p913vvj4dilu0; pc_333f7gpuishjximodvynnoisxujicgwaetzywgkxoxuje5ph3qyqjuid_onion__XSRF-TOKEN=eyJpdiI6IndpMlZNRGYyaDJuU0c0cUVVekNuUkE9PSIsInZhbHVlIjoiWlplRlZMakhBN21cL2FCckhtdFwvVUEzU2pSMUV3UWJQWXVzRmZVYW05cTdiQlZSZ3BuNTNKb25VQzBPcW11RE1BbDBSRzEyUUMyOEI0T1JiM3p0XC9ZdXFKUVZQYjk0aGpqdTc5SWppUytvdVZqNEljaU9zcG1RMWt5VlNoT29wMlEiLCJtYWMiOiJlYWY3YzNjNjg2YzQwOWY4ZGJjNjkxZTAyNzQ0NWVkNWY5NWYwYzhkNTE2NDNlZjkzNWEwNmI5OTdmODlkNzY2In0%3D; pc_333f7gpuishjximodvynnoisxujicgwaetzywgkxoxuje5ph3qyqjuid_onion__hr4ujvby8ds459og4kzcpbzwjdj_session=eyJpdiI6IjE4V0hOMXdxeURJblJYWXBUZTJNOEE9PSIsInZhbHVlIjoiZ0Vta3BqWllwQkZESWppNGp2YktZRGVUYTZLcEgzZUExdGRFdmFSc2kzaytXQ3NlSFVqWjBFZmJORGZGUFJWMW52eU1UckJNVHVHRFJIdlBndWh3R0lNdjZzeFBtKzBzc1g3czlHTjcwd3FNQW1iejl3d2F5TzdcLzcwXC9kMEVwOCIsIm1hYyI6IjJmMjNiN2VmN2JkMGI0ODkxNGY4NGJjMThkZjI2Njg3MjJhMzMwOTJhYjI5OGEwOWU5NmVhZjVjMzlmM2JiNmYifQ%3D%3D"

# Takes the relevant info of the copied cookies and puts it in the correct format automatically.
def convert_pasted_cookies_to_usable_format(pasted_cookies):
    all_cookies_str = pasted_cookies.split(':')[1] # to ignore the prefix "Cookie: ")
    individual_cookies = all_cookies_str.split(';')
    cookies_set = {}
    for cookie in individual_cookies:
        cookies_set.update({cookie.strip().split('=')[0]: cookie.strip().split('=')[1]})
    print(cookies_set)
    return cookies_set

cookies_copied = input()
set_of_cookies = convert_pasted_cookies_to_usable_format(cookies_copied)


{'PHPSESSID': '17gla03n2st3jj1chcv3i1i603', 'pc_333f7gpuishjximodvynnoisxujicgwaetzywgkxoxuje5ph3qyqjuid_onion__XSRF-TOKEN': 'eyJpdiI6ImNJRHg3MWVjYlArQTZHRjlXR0tqK0E9PSIsInZhbHVlIjoiYlZFRVdQZXZlelg0ZW9ieHNlR2liVXJ4ZHA3bWJxOFNcLzQ5dVloVEpaN2RYU2Q2bXRSN0NaVVFcL0M5emFid0dcL2tHNVh0XC9pUEtGUTM0RTNUNCtSeDhKVGxFblhtNnhpRG54eWNvVDBJZm5ZSTFCbmNGZDFzVUN4N1VqUDR4cndNIiwibWFjIjoiZDg4OWY5ZmU1ODFjZDU3MWVlNjJjY2E3Nzk5ZmY5ODViM2FlNmU1ZDVjNWVjZTY2YThmMGZlMzVjZjBkMjIzZCJ9', 'pc_333f7gpuishjximodvynnoisxujicgwaetzywgkxoxuje5ph3qyqjuid_onion__hr4ujvby8ds459og4kzcpbzwjdj_session': 'eyJpdiI6ImEyYUd5cFpwcmdkT0xNa3VIRkxtakE9PSIsInZhbHVlIjoiVkQ1SERyXC9zZXNZUzZjMXZRRzBWR3FFRVROa09ONHZ5UjQ3bWVnWnZCMkpzYzE4RlwvZVFmaVRTaWxuT1hPOUFvRHA5NnB0MzNLVHVpQ3FaTEpDb2x4MWFFUGxTSWlaMkxPZzJHRUkwZ2VVR3ZwZWJSamNsWGE4MGRHXC9pamsxXC9cLyIsIm1hYyI6ImZhMjFhZTIzOTRkMWM3NjU3MWJlYzU2MTMxZmY5YjRjNTRhY2ZhMjM0ODY2YzYzYTVjMmM2ZDZmY2ExYTkxMjEifQ%3D%3D'}


In [25]:

# Get the page category drugs
web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
print(web_page)
soup = BeautifulSoup(web_page.content, 'html.parser')
# response should be 200
print(web_page)



ConnectionError: SOCKSHTTPConnectionPool(host='mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion', port=80): Max retries exceeded with url: /items/category/drugs-and-chemicals (Caused by NewConnectionError('<urllib3.contrib.socks.SOCKSConnection object at 0x0000012D340D9AF0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [45]:
# create empty lists
vendor = []
vendor_offers = []
all_titles = []
all_quantities = []
all_prices = []
category = []
category_level_1 = []
category_level_2 = []
category_level_3 = []
ship_from = []
ship_to =[]



# crawl and scrape through pages
pages = []
for a in soup.find_all('a'):#,attrs={"class" : "page-link"}):
    print(a)
    pages.append(a.text)

# last_page = 2 #pages[-2]

<a href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/profile/missa00">
                                        My Profile
                                    </a>
<a href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/settings/profile">
                                        Edit Profile
                                    </a>
<a href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/settings/2fa">
                                        Account Security
                                    </a>
<a href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/wallet/change-pin">
                                        Change PIN
                                    </a>
<a href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/settings/news-feed">
                                        News Feed
                                    </a>
<a class="nav-link" href="http://mmd32xdcmzrd

In [46]:
last_page=966
for page in range(1, int(last_page)+1): #int(last_page)+1
    print(page, int(last_page))
    # NIET AANPASSEN
    url = 'http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/category/drugs-and-chemicals?order=none&vendors=all&shipping_to=any&shipping_from=any&product_type=any&payments_method=any&page=' + str(page)

    web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
    soup = BeautifulSoup(web_page.content, 'html.parser')

    for a in soup.find_all('a', href=True, class_='title'):
        print(a)
        all_titles.append(a.text.strip())

    for price in soup.find_all('span', class_='Price'):
        all_prices.append(re.findall("\d+\.\d+", price.text.strip())[0])

    for a in soup.find_all('div',attrs={"class" : "w-100 mt-3 small"}):
        if len(a.find_all('a'))==4:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(a.find_all('a')[3].text.strip())
        elif len(a.find_all('a'))==3:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(np.nan)
        elif len(a.find_all('a'))==2:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(np.nan)
            category_level_3.append(np.nan)
        last_item = a.find_all('a')[-1]
        category.append(last_item.text.strip())


    for tag_elm in soup.find_all('div', class_='shipping'):
        if tag_elm.find('span', class_='shippingFrom') != None:
            ship_from.append(tag_elm.find('span', class_='shippingFrom').text.strip())
        else:
            ship_from.append(np.nan)

    for tag_elm in soup.find_all('div', class_='shipping'):
        if tag_elm.find('span', class_='shippingTo') != None:
            ship_to.append(tag_elm.find('span', class_='shippingTo').text.strip())
        else:
            ship_to.append(np.nan)

    for a in soup.find_all('a', class_='font-weight-bold'):
        vendor.append(a.text.strip())




results = (list(zip(vendor, category_level_1, category_level_2, category_level_3, category, all_titles,
                    all_prices, ship_from, ship_to)))

df = pd.DataFrame(results,columns=['vendor', 'category_level_1', 'category_level_2', 'category_level_3',
                                   'highest_category', 'product', 'price in $', 'shipping_from', 'shipping_to'])
print(df)
df.to_csv('/home/rots/PycharmProjects/data-forensics/new_scrape.csv', index=False)

1 966
<a class="title" href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/35g-white-crumble-live-resin">3.5G WHITE CRUMBLE LIVE RESIN</a>
<a class="title" href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/5g-runtz-filtre3-by-barneys-farms">5G Runtz Filtré*3 by "Barneys Farms"</a>
<a class="title" href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/prestigeworldwide-1g-premium-grade-fish-scale-coke">PrestigeWorldwide 1G PREMIUM GRADE FISH SCALE COKE</a>
<a class="title" href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/royal-gorilla-hash-high-quality-a-5-g">Royal Gorilla Hash High Quality A+++ 5 G</a>
<a class="title" href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/500-x-150ug-wizard-lsd">500 x 150ug Wizard LSD</a>
<a class="title" href="http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/84g-blueberry-taho

In [47]:


results = (list(zip(vendor, category_level_1, category_level_2, category_level_3, category, all_titles,
                    all_prices, ship_from, ship_to)))

df = pd.DataFrame(results,columns=['vendor', 'category_level_1', 'category_level_2', 'category_level_3',
                                   'highest_category', 'product', 'price in $', 'shipping_from', 'shipping_to'])
print(df)
df.to_csv('/home/rots/PycharmProjects/data-forensics/scrape20220117.csv', index=False)

                 vendor     category_level_1 category_level_2  \
0        TheIcePrincess  Drugs and Chemicals              NaN   
1          WeedShopHulk  Drugs and Chemicals  Cannabis & Hash   
2      prestigeworldwde  Drugs and Chemicals       Stimulants   
3         MedicalCenter  Drugs and Chemicals  Cannabis & Hash   
4          MrImpossible  Drugs and Chemicals     Psychedelics   
...                 ...                  ...              ...   
19284         bioweedit  Drugs and Chemicals  Cannabis & Hash   
19285    fastpharmacyuk  Drugs and Chemicals           Benzos   
19286    fastpharmacyuk  Drugs and Chemicals          Opiates   
19287    fastpharmacyuk  Drugs and Chemicals       Stimulants   
19288    fastpharmacyuk  Drugs and Chemicals          Opiates   

      category_level_3     highest_category  \
0                  NaN  Drugs and Chemicals   
1                 Hash                 Hash   
2              Cocaine              Cocaine   
3       Buds & Flowers       Bu

In [44]:


results = (list(zip(vendor, category_level_1, category_level_2, category_level_3, category, all_titles,
                    all_prices, ship_from, ship_to)))

df = pd.DataFrame(results,columns=['vendor', 'category_level_1', 'category_level_2', 'category_level_3',
                                   'highest_category', 'product', 'price in $', 'shipping_from', 'shipping_to'])
print(df)
df.to_csv('/home/rots/PycharmProjects/data-forensics/new_scrape.csv', index=False)

                vendor     category_level_1     category_level_2  \
0       TheIcePrincess  Drugs and Chemicals                  NaN   
1         WeedShopHulk  Drugs and Chemicals      Cannabis & Hash   
2     prestigeworldwde  Drugs and Chemicals           Stimulants   
3        MedicalCenter  Drugs and Chemicals      Cannabis & Hash   
4         MrImpossible  Drugs and Chemicals         Psychedelics   
...                ...                  ...                  ...   
2515   LucySkyDiamonds  Drugs and Chemicals         Psychedelics   
2516         bestgroup  Drugs and Chemicals             Steroids   
2517           roidhub  Drugs and Chemicals             Steroids   
2518            drkend  Drugs and Chemicals  Prescriptions Drugs   
2519            mas220  Drugs and Chemicals         Psychedelics   

     category_level_3     highest_category  \
0                 NaN  Drugs and Chemicals   
1                Hash                 Hash   
2             Cocaine              Cocaine   

In [34]:
import socks
import socket
import requests
from bs4 import BeautifulSoup
import time
import re
import numpy as np
import pandas as pd

socks.set_default_proxy(socks.SOCKS5, "localhost", 9050) # one source said port 9150, Georgia's code used 9050
socket.socket = socks.socksocket

# These lines are copied from the Linkedin post.
def getaddrinfo(*args):
    return [(socket.AF_INET, socket.SOCK_STREAM, 6, '', (args[0], args[1]))]
socket.getaddrinfo = getaddrinfo

# ToRReZ market URL DEZE LINKE GEBRUIKEN OM DE COOCKIES TE PAKKEN
url = "http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/category/drugs-and-chemicals"


# Manually selected from the browser header after solving the captcha
cookies_copied = '''"pc_333f7gpuishjximodvynnoisxujicgwaetzywgkxoxuje5ph3qyqjuid_onion__hr4ujvby8ds459og4kzcpbzwjdj_session": "eyJpdiI6InlhazVNQzdVRmlRTFwvTnoyN1wvNEIxQT09IiwidmFsdWUiOiJSaU12Z1J0b2xSTjlNaittY0pSMEY2N1pEN1g1eXdKb1NQeU9DalRpaUY2cW43a0owQ1c2XC9SRm5HT3BCU1VLOStuOVhwRkNrOHltY3NoRkxZaWdBdE81b0lzbFM2K0xVeTBCcjlCYzR5WFYzblBTeGgxa1VzejdYandQWHZMWHYiLCJtYWMiOiJiNTBmNzNmZjMyMDJiZjA3NDYwYTA3NzkzZjNiMzQxMGY2MzRiNmFkOWM4NmE1NzE2ZDQ1ZjVkYWNiZjg0Yzk4In0=",
	"pc_333f7gpuishjximodvynnoisxujicgwaetzywgkxoxuje5ph3qyqjuid_onion__XSRF-TOKEN": "eyJpdiI6IkFlUmV0NUdieHRNbWV0cEhwb0VZd0E9PSIsInZhbHVlIjoidW5VZzV4cnhFU2xJYVwvVXZwYmZIc3Q1RWZORTNqZWJncDZQTUpPZkgzc1F2aExqU1laVmdaWVJhYllyMnJtMzZlZUlrYmVOY3g2NFR3MG54ZzJ6eTZlaUlOXC91cCtENkoyXC9nZ0FTd1B1VURCNDkzaFY5Q3BFZkVuRHZFSnFlY0giLCJtYWMiOiJlOWRhM2FjNTRkYjNkYTA1YjIxYzdmMzRkNGNmOGE1ZWY4OTFjMTNhYjkxN2JkYzk2YWVmY2ZmOGM5ZjQ3NWZlIn0=",
	"PHPSESSID": "vhra8tc6od8joutjkhis3leoh6"'''

# Takes the relevant info of the copied cookies and puts it in the correct format automatically.
def convert_pasted_cookies_to_usable_format(pasted_cookies):
    all_cookies_str = pasted_cookies.split(':')[1] # to ignore the prefix "Cookie: ")
    individual_cookies = all_cookies_str.split(';')
    cookies_set = {}
    for cookie in individual_cookies:
        cookies_set.update({cookie.strip().split('=')[0]: cookie.strip().split('=')[1]})
    return cookies_set


s = requests.Session()
s.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0'
})

proxies = {
    'http': 'socks5h://localhost:9050',
    'https': 'socks5h://localhost:9050'
}

set_of_cookies = convert_pasted_cookies_to_usable_format(cookies_copied)

# Get the page category drugs
web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
soup = BeautifulSoup(web_page.content, 'html.parser')



# create empty lists
vendor = []    
vendor_offers = []
all_titles = []
all_quantities = []
all_prices = []
category = []
category_level_1 = []
category_level_2 = []
category_level_3 = []
ship_from = []
ship_to =[]



# crawl and scrape through pages
pages = []
for a in soup.find_all('a',attrs={"class" : "page-link"}):
    print(a)
    pages.append(a.text)
last_page = 2 #pages[-2]

for page in range(1, int(last_page)+1): #int(last_page)+1
    print(page, int(last_page))
    # NIET AANPASSEN
    url = 'http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/items/category/drugs-and-chemicals' + str(page)

    web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
    soup = BeautifulSoup(web_page.content, 'html.parser')


    for a in soup.find_all('a', href=True, class_='title'):
        all_titles.append(a.text.strip())

    for price in soup.find_all('span', class_='Price'):
        all_prices.append(re.findall("\d+\.\d+", price.text.strip())[0])
        
    for a in soup.find_all('div',attrs={"class" : "w-100 mt-3 small"}):
        if len(a.find_all('a'))==4:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(a.find_all('a')[3].text.strip())
        elif len(a.find_all('a'))==3:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(np.nan)
        elif len(a.find_all('a'))==2:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(np.nan)
            category_level_3.append(np.nan)
        last_item = a.find_all('a')[-1]
        category.append(last_item.text.strip())

           
    for tag_elm in soup.find_all('div', class_='shipping'):
        if tag_elm.find('span', class_='shippingFrom') != None:
            ship_from.append(tag_elm.find('span', class_='shippingFrom').text.strip())
        else:
            ship_from.append(np.nan)

    for tag_elm in soup.find_all('div', class_='shipping'):
        if tag_elm.find('span', class_='shippingTo') != None:
            ship_to.append(tag_elm.find('span', class_='shippingTo').text.strip())
        else:
            ship_to.append(np.nan)

    for a in soup.find_all('a', class_='font-weight-bold'):
        vendor.append(a.text.strip())


            
            
results = (list(zip(vendor, category_level_1, category_level_2, category_level_3, category, all_titles, 
                    all_prices, ship_from, ship_to)))



df = pd.DataFrame(results,columns=['vendor', 'category_level_1', 'category_level_2', 'category_level_3', 
                                   'highest_category', 'product', 'price in $', 'shipping_from', 'shipping_to'])    


1 2
2 2


In [33]:


df = pd.DataFrame(results,columns=['vendor', 'category_level_1', 'category_level_2', 'category_level_3',
                                   'highest_category', 'product', 'price in $', 'shipping_from', 'shipping_to'])

df

,vendor,category_level_1,category_level_2,category_level_3,highest_category,product,price in $,shipping_from,shipping_to
0,TheIcePrincess,Drugs and Chemicals,NaN,NaN,Drugs and Chemicals,3.5G WHITE CRUMBLE LIVE RESIN,75.00,United States,United States
1,WeedShopHulk,Drugs and Chemicals,Cannabis & Hash,Hash,Hash,"5G Runtz Filtré*3 by ""Barneys Farms""",102.93,France,France
2,prestigeworldwde,Drugs and Chemicals,Stimulants,Cocaine,Cocaine,PrestigeWorldwide 1G PREMIUM GRADE FISH SCALE ...,323.72,Australia,Australia
3,MedicalCenter,Drugs and Chemicals,Cannabis & Hash,Buds & Flowers,Buds & Flowers,Royal Gorilla Hash High Quality A+++ 5 G,66.60,Germany,Europe
4,MrImpossible,Drugs and Chemicals,Psychedelics,LSD,LSD,500 x 150ug Wizard LSD,504.94,Australia,Australia
5,fadedfork,Drugs and Chemicals,Cannabis & Hash,Buds & Flowers,Buds & Flowers,84g blueberry tahoe x gsc,931.53,United Kingdom,Worldwide
6,RealNarcotiks3,Drugs and Chemicals,Stimulants,Speed,Speed,50 ICE SPEED PILLS 25-30 MG+ EXTRA 10% ADDED,69.95,NaN,North America
7,jimmjoness,Drugs and Chemicals,Ecstasy,MDMA,MDMA,1 Kilo MDMA CHAPMAGNE CRISTALS,632.90,NaN,Worldwide
8,drkend,Drugs and Chemicals,NaN,NaN,Drugs and Chemicals,Buy ADB-Pinaca Online (250 G),750.00,China,Worldwide
9,fadedfork,Drugs and Chemicals,Cannabis & Hash,Buds & Flowers,Buds & Flowers,28g jaffa caked cookies - grade b,254.05,United Kingdom,Worldwide


In [8]:
df_vendor = pd.read_csv("./scrape20220117.csv")
vendor = df_vendor['vendor']
vendor

0          TheIcePrincess
1            WeedShopHulk
2        prestigeworldwde
3           MedicalCenter
4            MrImpossible
               ...       
19284           bioweedit
19285      fastpharmacyuk
19286      fastpharmacyuk
19287      fastpharmacyuk
19288      fastpharmacyuk
Name: vendor, Length: 19289, dtype: object

## Crawl and Scrape all vendor information

In [9]:
unique_vendors = list(dict.fromkeys(vendor))

vendor_page = []

for vendor in unique_vendors:
    print(vendor)
    vendor_page.append("http://mmd32xdcmzrdlpoapkpf43dxig5iufbpkkl76qnijgzadythu55fvkqd.onion/profile/" + str(vendor))

since = [] 
transactions = []
feedback_total = []
feedback_positive = []
feedback_negative = []
disputes_total =[]
disputes_won = []
disputes_lost = []
finalize_early = []
rank = []
verification = []

all_variables = [since, transactions, feedback_total, feedback_positive, 
                 feedback_negative, disputes_total, disputes_won, disputes_lost, finalize_early]
all_text = ['On ToRReZ Market Since:', 'Total Amount Of Transactions', 'Total Feedback Received', 
            'Positive Feedback Received Ratio', 'Negative Feedback Received Ratio', 'Disputes Total:', 
            'Disputes Won:', 'Disputes Lost:', 'Finalize Early']    
    
for page in vendor_page:
    web_page = s.get(page, cookies=set_of_cookies, proxies=proxies)
    soup = BeautifulSoup(web_page.content, 'html.parser')
    
    for numb in range(0,len(all_variables)):
        for i in soup.find_all('table', class_='table table-borderless table-sm mt-1 table-fixed'):
            value = soup.find('th', text=all_text[numb])
            # all_variables[numb].append(value.find_next_sibling('td').text.strip())

    for span in soup.find_all('span', class_='badge badge-success question'):
        rank.append(span.text.strip())
        
    for tag_elm in soup.find_all('div', class_='col singleItemDetails mt-2'):
        if tag_elm.find('a', class_='badge badge-dark') != None:
            verification.append(tag_elm.find('a', class_='badge badge-dark').text.strip())
        else:
            verification.append(np.nan)


TheIcePrincess
WeedShopHulk
prestigeworldwde
MedicalCenter
MrImpossible
fadedfork
RealNarcotiks3
jimmjoness
drkend
DutchGlory
original_kimono
PremierLeagueWorldWide
Utopia
underlinecost
mdacanada
2GD4DN
GreyGooseGlobal
WA2018
gigglytities
StrainPirate
BestBudz
TheBodyShop
Mont-Blanc
MadeInFrance
thedopehouse
AlphaGear
RoyalMailer
LordOfWeed
SteroidWarehouse
thelab1
ScotchAndCoca
JAPAN_HIGH
cadillacblack
AmazonPrimePlus
SoldGold
DutchPharmacy
California420Service
SpaceInvaders
BwEmpire
dogreatthings
EUROSHOPPER
GreenBritain
drugpluguk
hollandgood
CanadianSmoker12
AlwaysOverweight
NarcoticsWorldwide
PlanetHollywood
WeedZilla777
Licensedbud
richhigh999
monoko
auspride
BcBudGod
Thegreens
DrugzDealerNRW
QUICKDRUGSUK
OPENLINE
Hasbro
saintpablooooo
AddamsFamily
exclusivepharma
cheshirecat82
TheodorMorell
drugsincuk
premierpharma
TheBestOfHolland
PenileFractures
stealthmedsuk
MisterX
lekkercocaine
ActiveJ
gratefullydead
dutchsuperior
PianoKeyz
TheRxPharma
COUNTYLINE
hollandgottalent
rainbowlan

In [10]:
results_2 = (list(zip(unique_vendors, rank, verification, since, transactions, feedback_total, feedback_positive, feedback_negative, 
                      disputes_total, disputes_won, disputes_lost, finalize_early)))



df_2 = pd.DataFrame(results_2,columns=['vendor', 'rank', 'verifcation', 'since', 'transactions', 'feedback_total', 
                                   'feedback_positive', 'feedback_negative','disputes_total',
                                   'disputes_won', 'disputes_lost', 'finalize_early'])    


In [11]:
df_2

,vendor,rank,verifcation,since,transactions,feedback_total,feedback_positive,feedback_negative,disputes_total,disputes_won,disputes_lost,finalize_early
